In [1]:
%%capture
! pip install ml-collections
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/WhenPigsFlyContext/baselines/gbvs

In [2]:
import sys
import cv2
import time
from matplotlib import pyplot as plt
from tqdm import tqdm, trange
import numpy as np
import pandas as pd
import pickle
import random
import copy

import os
import shutil
from PIL import Image, ImageDraw

import torch
from torch.utils.data import Dataset
from torchvision.transforms.functional import to_tensor, normalize
from torchvision import transforms

from saliency_models import gbvs, ittikochneibur
from naturaldesign.naturaldesign import NaturalDesign
from utils import *
sys.path.append("..")

In [3]:
test_imagedir = "../../naturaldesign/"
dataset = NaturalDesign(test_imagedir)

In [5]:
gbvs_res, size, img_size = [], 200, (1024, 1280)
gbvs_attention_map, scanpath = {}, {}

for id in trange(1, len(dataset)+1):

    img, bbox_relative = dataset[id]
    # get the target bounding box
    tg_loc = bbox_cordinates(bbox_relative, img_size[1], img_size[0])

    # compute the saliency map
    saliency_map_gbvs = gbvs.compute_saliency(img)

    saliency_map_gbvs_tensor = torch.from_numpy(saliency_map_gbvs).detach()
    saliency_map_gbvs_tensor = transforms.Resize(img_size)(saliency_map_gbvs_tensor.unsqueeze(0))

    gbvs_attention_map[id] = copy.deepcopy(saliency_map_gbvs_tensor)

    search_num, path = searchProcesswithPath(tg_loc, saliency_map_gbvs_tensor, img_size, size)
    scanpath[id] = path
    print(search_num)
    gbvs_res.append(search_num)

  0%|          | 1/240 [00:05<23:41,  5.95s/it]

24


  1%|          | 2/240 [00:10<21:20,  5.38s/it]

79


  1%|▏         | 3/240 [00:17<23:50,  6.04s/it]

...

19


100%|█████████▉| 239/240 [19:09<00:04,  4.94s/it]

20


100%|██████████| 240/240 [19:13<00:00,  4.81s/it]

6


In [6]:
gbvs_naturaldesign_res = {}
gbvs_naturaldesign_res['scanpath'] = scanpath
gbvs_naturaldesign_res['attention_map'] = gbvs_attention_map

In [7]:
with open("../results/NaturalDesign/[Fig5]NaturalDesign_gbvs_res.pkl", "wb") as tf:
    pickle.dump(gbvs_naturaldesign_res, tf)

In [ ]:
gbvs_accu_ND = model_performance(gbvs_res, len(gbvs_res))
gbvs_accu_ND[:11]

[0,
 0.1125,
 0.20833333333333334,
 0.3125,
 0.35833333333333334,
 0.4,
 0.4666666666666667,
 0.5083333333333333,
 0.5291666666666667,
 0.5583333333333333,
 0.6]

In [ ]:
with open("../results/NaturalDesign/naturaldesign_gbvs_accu_performance.pkl", "wb") as tf:
    pickle.dump(gbvs_accu_ND,tf)